In [1]:
import os
from os.path import join, dirname, getsize, exists
from glob import glob

import numpy as np
import pandas as pd

import nilearn
from nilearn.masking import apply_mask

from tqdm import tqdm

import mydef

/home/sungbeenpark/anaconda3/envs/GA/lib/python3.9/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
GA = mydef.GA()

OS : linux
Google Drive is detected!
Git directory is detected!


# ROIs

In [3]:
GA.roi_imgs = {}

1. Localizer

In [4]:
path_list = sorted(glob(join(GA.dir_mask,'localizer','n200_*.nii')))
temp = ['n200_c1_L_Postcentral'
        , 'n200_c2_R_CerebellumIV-V'
        , 'n200_c3_R_Postcentral'
        , 'n200_c4_L_Putamen'
        , 'n200_c5_R_SMA'
        , 'n200_c6_R_CerebellumVIIIb'
        , 'n200_c7_L_Thalamus']
list_ = []
for s in temp:
    list_.append(glob(join(GA.dir_mask, 'localizer', '%s*.nii'%s))[0])
for s in list_:
    roi = s.split('/')[-1].replace('_mask.nii', '')
    GA.roi_imgs[roi] = nilearn.image.load_img(s)

2. DMN

In [5]:
## DMN regions(global)
# module_names = ['Core', 'dMsub', 'MTLsub']

# for name in module_names:
#     DMN_list = glob(join(GA.dir_mask,'DMN',name+'.nii'))
#     for path in DMN_list:
#         fname = path.split('/')[-1].replace('.nii', '')
#         GA.roi_imgs[fname] = nilearn.image.load_img(path)
        
# ## DMN regions(detail)
module_names = ['Core', 'dMsub', 'MTLsub']

for name in module_names:
    DMN_list = glob(join(GA.dir_mask,'DMN',name+'_*.nii'))
    for path in DMN_list:
        fname = path.split('/')[-1].replace('.nii', '')
        GA.roi_imgs[fname] = nilearn.image.load_img(path)

3. Yeo's network #1

In [6]:
for idx in GA.fan_info[GA.fan_info.yeo_17network == 1].index:
    nn = GA.fan_info.loc[idx,'label']
    path = glob(join(GA.dir_mask, 'fan280', '*%03d*.nii.gz'%nn))[0]
#     region = GA.fan_info.loc[idx,'region']
    region = 'fan%03d'%(GA.fan_info.loc[idx,'label'])
    GA.roi_imgs[region] = nilearn.image.load_img(path)

In [7]:
# GA.roi_imgs

{'n200_c1_L_Postcentral': <nibabel.nifti1.Nifti1Image at 0x7f3d0d526280>,
 'n200_c2_R_CerebellumIV-V': <nibabel.nifti1.Nifti1Image at 0x7f3d0d526b20>,
 'n200_c3_R_Postcentral': <nibabel.nifti1.Nifti1Image at 0x7f3d0d52cca0>,
 'n200_c4_L_Putamen': <nibabel.nifti1.Nifti1Image at 0x7f3d0d52cb80>,
 'n200_c5_R_SMA': <nibabel.nifti1.Nifti1Image at 0x7f3d0d52c880>,
 'n200_c6_R_CerebellumVIIIb': <nibabel.nifti1.Nifti1Image at 0x7f3d0d52c5e0>,
 'n200_c7_L_Thalamus': <nibabel.nifti1.Nifti1Image at 0x7f3d0d52c310>,
 'Core_aMPFC_r': <nibabel.nifti1.Nifti1Image at 0x7f3d0d4c6460>,
 'Core_aMPFC_l': <nibabel.nifti1.Nifti1Image at 0x7f3d0d4c61f0>,
 'Core_PCC_r': <nibabel.nifti1.Nifti1Image at 0x7f3d0d4c6220>,
 'Core_PCC_l': <nibabel.nifti1.Nifti1Image at 0x7f3dd83eb6a0>,
 'dMsub_TempP_r_temp': <nibabel.nifti1.Nifti1Image at 0x7f3d0d526700>,
 'dMsub_dMPFC': <nibabel.nifti1.Nifti1Image at 0x7f3d0d526550>,
 'dMsub_TPJ_r': <nibabel.nifti1.Nifti1Image at 0x7f3d0d526970>,
 'dMsub_TPJ_l': <nibabel.nifti1.Nif

# Calculating

In [8]:
runs = ['r%02d'%(r+1) for r in range(6)]
list_ = []
for subj in GA.list_subj:
    for gg in ['GA', 'GB']:
        for run in runs:
            list_.append([gg, subj, run])
list_ = np.array(list_)
# list_

In [11]:
for gg, subj, run in tqdm(list_):
    path_data = join(GA.dir_stats, 'GLM.MO', subj
                     , '%s.bp_demean.errts.MO.%s.nii.gz'%(gg+subj, run))
    for region, mask in GA.roi_imgs.items():
        dir_output = join(GA.dir_stats, 'GLM.MO', 'tsmean', region)
        os.makedirs(output_dir, exist_ok=True)
        output = join(dir_output, 'tsmean.bp_demean.errts.MO.%s.%s.%s.1D'%(gg+subj,run,region))
        if not exists(output):
#             img = nilearn.image.load_img(path_data)
            masked_data = apply_mask(path_data, mask)
            

100%|██████████| 360/360 [00:00<00:00, 197948.27it/s]


In [9]:
region = 'n200_c1_L_Postcentral'
mask = GA.roi_imgs[region]
gg, subj, run = 'GA', '01', 'r01'
path_data = join(GA.dir_stats, 'GLM.MO', subj
                 , '%s.bp_demean.errts.MO.%s.nii.gz'%(gg+subj, run))

In [10]:
img = nilearn.image.load_img(path_data)

KeyboardInterrupt: 

In [ ]:
masked_img = GA.fast_masking

In [14]:
masked_data = apply_mask(path_data, mask)

error: Error -3 while decompressing data: invalid block type

In [16]:
dir_output = join(GA.dir_stats, 'GLM.MO', 'tsmean.backup', region)
output = join(dir_output, 'tsmean.bp_demean.errts.MO.%s.%s.%s.1D'%(gg+subj,run,region))

with open(output,'rb') as fr:
    tsmean = np.genfromtxt(fr, delimiter='\n')
    
tsmean.shape

(1096,)